In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import numpy as np
import pandas as pd
import xarray as xr

import jax
import jax.numpy as jnp
from jax.flatten_util import ravel_pytree
import jax.tree_util as jtu

from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

from jax.nn import SIREN, fit
from utils import pool_image, get_coords_for_image

import matplotlib.pyplot as plt
from multiprocessing import Pool

In [3]:
seed = 3
aq_data = xr.open_dataset("/home/patel_zeel/delhi_aq/data/delhi_cpcb_2022.nc").to_dataframe().reset_index().set_index("time", drop=False)
aq_data['time'] = aq_data['time'].astype(int)
aq_data['time'] = MinMaxScaler((-10, 10)).fit_transform(aq_data['time'].values.reshape(-1, 1)).reshape(-1)
aq_data = aq_data.dropna(subset="PM2.5")
aq_data.head(2)

,station,time,From Date,To Date,PM2.5,PM10,NO,NO2,NOx,NH3,...,RH,WS,WD,SR,BP,AT,RF,TOT-RF,latitude,longitude
time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:30:00,"Alipur, Delhi - DPCC",-10.000000,2022-01-01 00:00:00,2022-01-01 01:00:00,219.5,342.75,94.73,53.95,105.95,3.78,...,79.75,NaN,205.95,4.6,999.1,6.85,0.0,0.0,28.815329,77.15301
2022-01-01 01:30:00,"Alipur, Delhi - DPCC",-9.997723,2022-01-01 01:00:00,2022-01-01 02:00:00,224.5,360.25,89.30,44.70,96.60,5.50,...,82.23,NaN,206.05,4.6,998.9,6.72,0.0,0.0,28.815329,77.15301
